In [1]:
import MetaTrader5 as mt5
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
import mplfinance as mpf
from mplfinance import plot as mplot
import os
from tensorflow import keras
from matplotlib.image import imread
from datetime import date
import datetime
from datetime import datetime as dt
import matplotlib 
matplotlib.use('agg')
import matplotlib.pyplot as plt
import re
import tensorflow
import cv2 as cv
from tqdm import tqdm
from PIL import ImageFont, ImageDraw, Image
import gc
from datetime import datetime
import pytz
import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_position_sell(symbol, lot, sl , tp):
    symbol_info = mt5.symbol_info(symbol)
    point = mt5.symbol_info(symbol).point
    price = mt5.symbol_info_tick(symbol).bid
    deviation = 20
    print(price + (sl * point))
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_SELL,
        "price": price,
        "sl": price + (sl * price),
        "tp": price - (tp * price),
        "deviation": deviation,
        "comment": "python script open",
        "type_filling":mt5.SYMBOL_TRADE_EXECUTION_INSTANT,
    }
    result = mt5.order_send(request)
    return result

In [3]:
def get_position_buy(symbol, lot, sl , tp):
    symbol_info = mt5.symbol_info(symbol)
    point = mt5.symbol_info(symbol).point
    price = mt5.symbol_info_tick(symbol).bid
    deviation = 20
    print(price + (sl * point))
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_BUY,
        "price": price,
        "sl": price - (sl * price),
        "tp": price + (tp * price),
        "deviation": deviation,
        "comment": "python script open",
        "type_filling":mt5.SYMBOL_TRADE_EXECUTION_INSTANT,
    }
    result = mt5.order_send(request)
    return result

In [4]:
# currency_pairs= ['AUDCAD', 'AUDCHF','AUDJPY','AUDNZD','AUDUSD','CADCHF','CADJPY','CHFJPY',
#                  'EURAUD','EURCAD','EURCHF','EURGBP','EURJPY','EURNZD','EURUSD',
#                  'GBPAUD','GBPCAD','GBPCHF','GBPJPY','GBPNZD','GBPUSD','NZDJPY',
#                  'NZDCAD','NZDCHF','NZDUSD','USDCAD','USDCHF','USDJPY']

In [5]:
currency_pairs= ['AUDCAD', 'AUDCHF','AUDJPY','AUDNZD','AUDUSD','CADCHF','CADJPY','CHFJPY',
                 'EURAUD','EURCAD','EURCHF','EURGBP','EURJPY','EURNZD','EURUSD',
                 'GBPAUD','GBPCAD','GBPCHF','GBPJPY','GBPNZD','GBPUSD','NZDJPY',
                 'NZDCAD','NZDCHF','NZDUSD','USDCAD','USDCHF','USDJPY']

In [6]:
# connect to MetaTrader 5
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

mt5.account_info()

AccountInfo(login=5386640, trade_mode=0, leverage=500, limit_orders=0, margin_so_mode=0, trade_allowed=True, trade_expert=True, margin_mode=2, currency_digits=2, fifo_close=False, balance=10605.07, credit=0.0, profit=-256.63, equity=10348.44, margin=1542.67, margin_free=8805.77, margin_level=670.8135894261248, margin_so_call=50.0, margin_so_so=20.0, margin_initial=0.0, margin_maintenance=0.0, assets=0.0, liabilities=0.0, commission_blocked=0.0, name='Zahed Zahedi', server='AMarkets-Demo', currency='USD', company='AMarkets')

In [7]:
image_path_1h = r'C:\Users\Administrator\Desktop\Trade_1005\Images_1h'
image_path_4h = r'C:\Users\Administrator\Desktop\Trade_1005\Images_4h'
RecolPath_1h = r'C:\Users\Administrator\Desktop\Trade_1005\Images_1h\Recolored'
RecolPath_4h = r'C:\Users\Administrator\Desktop\Trade_1005\Images_4h\Recolored'
model_1h= tensorflow.keras.models.load_model('ckpt_03_CorrectHighLowWithoutResize.hdf5')
model_4h_old= tensorflow.keras.models.load_model('ckpt_04_4h_best.hdf5')
model_4h_new= tensorflow.keras.models.load_model('ckpt_04_tp008sl003_best_4h.hdf5')
df_minmax_1h= pd.read_csv('df_minmax_mrged_1h.csv')
df_minmax_4h= pd.read_csv('df_minmax_merged_4h.csv')
tesh_optimum_1h = pd.read_csv('df_best_final_EmbedRecolor_withoutResize_CorrectHL.csv')
tesh_optimum_4h = pd.read_excel('Best_TreshFor_4h_Merged.xlsx')
# Selected_pairs_for_Old_model_4h= ['CHFJPYb', 'AUDUSDb', 'GBPCADb', 'EURJPYb', 'GBPJPYb', 'EURNZDb', 'EURGBPb',
#                                   'NZDUSDb', 'AUDCHFb', 'AUDCADb', 'NZDCHFb', 'EURCADb', 'EURUSDb', 'GBPUSDb', 'NZDCADb']
# Selected_pairs_for_New_model_4h= ['USDCADb', 'EURAUDb', 'EURCHFb', 'GBPAUDb', 'CADJPYb', 'AUDNZDb',
#                                   'USDJPYb', 'CADCHFb', 'GBPCHFb', 'AUDJPYb', 'GBPNZDb', 'NZDJPYb', 'USDCHFb']

In [8]:
Selected_pairs_for_Old_model_4h= ['CHFJPY', 'AUDUSD', 'GBPCAD', 'EURJPY', 'GBPJPY', 'EURNZD', 'EURGBP',
                                  'NZDUSD', 'AUDCHF', 'AUDCAD', 'NZDCHF', 'EURCAD', 'EURUSD', 'GBPUSD', 'NZDCAD']
Selected_pairs_for_New_model_4h= ['USDCAD', 'EURAUD', 'EURCHF', 'GBPAUD', 'CADJPY', 'AUDNZD',
                                  'USDJPY', 'CADCHF', 'GBPCHF', 'AUDJPY', 'GBPNZD', 'NZDJPY', 'USDCHF']

In [9]:
mc = mpf.make_marketcolors(up='g',down='r')
s  = mpf.make_mpf_style(marketcolors=mc)

In [10]:
def make_candle_stick(df, image_path, pair):
    df['time']= df.time.apply(lambda x: dt.utcfromtimestamp(x).strftime('%Y%m%d %H%M%S'))
    df['time']= pd.to_datetime(df['time'])
    time_log= df.iloc[48]['time']
    
    openn = df.iloc[48]['open']
    high = df.iloc[48]['high']
    low = df.iloc[48]['low']
    close = df.iloc[48]['close']
    df = df.set_index('time')
    df.rename(columns={'tick_volume':'volume'}, inplace=True)
    
    HighLow= str(np.max(df.iloc[1:, 1])/np.min(df.iloc[1:, 2])).replace('.', '')[:6]
    if '_' in HighLow:
        HighLow = HighLow.replace('_', '0')
    
    #Heiken Ashi
    df_temp_Ha = df.copy()
    for i in range(1, len(df_temp_Ha)):
        df_temp_Ha.iloc[i, 0] = (df_temp_Ha.iloc[i-1, 0] + df_temp_Ha.iloc[i-1, 3])/2
        df_temp_Ha.iloc[i, 3] = (df_temp_Ha.iloc[i, 0] + df_temp_Ha.iloc[i, 1] + df_temp_Ha.iloc[i, 2] + df_temp_Ha.iloc[i, 3])/4
    
    time_log = str(time_log).replace(':', '').replace(' ', '').replace('-', '')
    
    img_name = "".join([pair, '_' ,str(time_log),'_', HighLow, '.jpg'])
    img_path= os.path.join(image_path, img_name)
    
    mplot(df_temp_Ha.iloc[1:-1, :], type='candle', volume=False, figscale=2,figratio=(10,10), style=s, savefig=img_path)
    return time_log, img_path, openn, high, low, close


In [11]:
# font = ImageFont.truetype('times.ttf', 20)
font = cv.FONT_HERSHEY_COMPLEX
loc= (600, 795)
font_scale= 1.1
color = (0, 0, 0)
thickness= 3

# print(df_minmax)

In [12]:
def makeEmbedImage(img_name, RecolPath, df_minmax, pair):
    img = cv.imread(img_name)
    img = img[140:-207, 220: -120, :]
    hl = int(img_name.split('.')[0].split('_')[-1])
    if hl < 100:
        hl = hl * 10000
    elif hl < 1000:
        hl = hl * 1000
    elif hl < 10000:
        hl = hl * 100
    elif hl < 100000:
        hl = hl * 10

    mx = df_minmax.loc[0, pair]
    mn = df_minmax.loc[1, pair]
    hl = (hl - mn) / (mx - mn)
    w = np.uint8(((hl * 255)))
    img[771:850, 20:550, 2] = w
    img[771:850, 20:550, 0] = w
    img[771:850, 20:550, 1] = 0
    img = cv.putText(img, pair, loc, font, font_scale, color, thickness, cv.LINE_AA)
    img = cv.resize(img, (300, 300))
    name= img_name.split('\\')[-1]
    cv.imwrite(os.path.join(RecolPath, name), img)
    return img


In [13]:
str(dt.now()), str(dt.now()).split(':')[0].split(' ')[-1]

('2022-01-19 18:41:18.700429', '18')

In [14]:
# tp= tesh_optimum_4h.loc[tesh_optimum_4h['p']== 'EURUSD','t'].values[0]
# sl= tesh_optimum_4h.loc[tesh_optimum_4h['p']== 'EURUSD','s'].values[0]
# acc_u= tesh_optimum_4h.loc[tesh_optimum_4h['p']== 'EURUSD','u'].values[0]
# acc_d= tesh_optimum_4h.loc[tesh_optimum_4h['p']== 'EURUSD','d'].values[0]
# tp, sl, acc_u, acc_d

In [15]:
def make_predict():
    list_log_dashboard= []
    list_log = []
    for pair in currency_pairs:
        tp= tesh_optimum_4h.loc[tesh_optimum_4h['p']== pair,'t'].values[0]
        sl= tesh_optimum_4h.loc[tesh_optimum_4h['p']== pair,'s'].values[0]
        acc_u= tesh_optimum_4h.loc[tesh_optimum_4h['p']== pair,'u'].values[0]
        acc_d= tesh_optimum_4h.loc[tesh_optimum_4h['p']== pair,'d'].values[0]

        rates_1h = mt5.copy_rates_from_pos(pair, mt5.TIMEFRAME_H1, 0, 50)
        df_1h=pd.DataFrame(rates_1h).reset_index(drop=True)
        time_log, img_name_1h, open_1h, high_1h, low_1h, close_1h = make_candle_stick(df_1h, image_path_1h, pair)
        img_1h= makeEmbedImage(img_name_1h, RecolPath_1h, df_minmax_1h, pair)
        acc_1h=model_1h.predict(np.array([img_1h,img_1h]))[0,1]

        rates_4h = mt5.copy_rates_from_pos(pair, mt5.TIMEFRAME_H4, 0, 50)
        df_4h=pd.DataFrame(rates_4h).reset_index(drop=True)
        time_log, img_name_4h, open_4h, high_4h, low_4h, close_4h = make_candle_stick(df_4h, image_path_4h, pair)
        img_4h= makeEmbedImage(img_name_4h, RecolPath_4h, df_minmax_4h, pair)
        if pair in Selected_pairs_for_Old_model_4h:
            acc_4h=model_4h_old.predict(np.array([img_4h,img_4h]))[0,1]
        else:
            acc_4h=model_4h_new.predict(np.array([img_4h,img_4h]))[0,1]
        print(f'{pair}, acc_1h:{acc_1h}, acc_4h:{acc_4h}')

        if acc_1h >= acc_u:
            print('------ Buy Position -------')
            position=get_position_buy(pair, 0.3,sl , tp)
            positions[position.order]={"pair":pair, "hour":1}
#             df_positions = pd.DataFrame(positions)
#             df_positions.to_csv('df_positions.csv', index= False)
        elif acc_1h <= acc_d:
            print('--------- Sell Position --------')
            position=get_position_sell(pair, 0.3,sl , tp)
            positions[position.order]={"pair":pair, "hour":1}
#             df_positions = pd.DataFrame(positions)
#             df_positions.to_csv('df_positions.csv', index= False)

        list_log.append([time_log, pair, img_name_1h, img_name_4h, acc_1h, acc_4h,open_1h, high_1h, low_1h, close_1h, open_4h, high_4h, low_4h, close_4h])
        list_log_dashboard.append([pair, acc_1h, acc_4h])

        if pair == currency_pairs[-1]:
            arr_log_dashboard = np.array(list_log_dashboard)
            arr_log_dashboard= arr_log_dashboard.reshape(int(len(currency_pairs)), 3)
            df_dashboard= pd.DataFrame(arr_log_dashboard, columns=['PAIR','1H', '4H'])
            df_dashboard.to_excel(r'C:\xampp\htdocs\fx_dashboard\pred.xlsx')



            arr_log = np.array(list_log)
            arr_log= arr_log.reshape(int(len(currency_pairs)), 14)
            df_temp = pd.DataFrame(arr_log, columns=['time_log','pair', 'img_name_1h','img_name_4h', 'acc_1h',
                                                     'acc_4h','open_1h', 'high_1h',  'low_1h','close_1h',
                                                    'open_4h','high_4h', 'low_4h', 'close_4h'])
    return df_temp

In [16]:
positions={}
i=0
while(True):
    time = dt.now()
    min_part= str(time).split(':')[1]
    sec_part= str(time).split(':')[-1].split('.')[0]
    if (min_part == '59') & (int(sec_part) >= 55):
#         if 'df_positions.csv' in os.listdir(r'C:\Users\Administrator\Desktop\Trade_1005'):
#             positions = dict(pd.read_csv('df_positions.csv'))
        
        for position in list(positions.keys()):
            positions[position]["hour"]+=1
            if positions[position]["hour"]==24:
                try:
                    mt5.Close(positions[position]["pair"],ticket=position)
                except:
                    pass
                positions.pop(position, None)
#             df_positions = pd.DataFrame(positions)
#             df_positions.to_csv('df_positions.csv', index= False)  
        print(f'---- {time} ------ ')
        try:
            df_temp= make_predict()
            file_name= r'C:\Users\Administrator\Desktop\Trade_1005\Logs_1005/df_log_{0}_5.csv'.format(str(i))
            if i == 0:
                df_temp.to_csv('df_log.csv', index= False)
                df_temp.to_csv(file_name, index= False)
            else:
                df_log= pd.read_csv('df_log.csv')
                df_log = pd.concat([df_log, df_temp], axis = 0)
                df_log.to_csv('df_log.csv', index= False)
                df_log.to_csv(file_name, index= False)
            i+=1
        except Exception as e:
            print(e)
            

---- 2022-01-19 18:59:55.015197 ------ 
AUDCAD, acc_1h:0.22161811590194702, acc_4h:0.24770957231521606
AUDCHF, acc_1h:0.0013749145437031984, acc_4h:0.36944159865379333
--------- Sell Position --------
0.65810006
AUDJPY, acc_1h:0.015345045365393162, acc_4h:0.04034361243247986
AUDNZD, acc_1h:0.0008853660547174513, acc_4h:0.5198695063591003
AUDUSD, acc_1h:0.1334189772605896, acc_4h:0.29556044936180115
CADCHF, acc_1h:0.20907868444919586, acc_4h:0.7035027742385864
CADJPY, acc_1h:0.13013754785060883, acc_4h:0.523475170135498
CHFJPY, acc_1h:0.14192357659339905, acc_4h:0.3622513711452484
EURAUD, acc_1h:0.062056299299001694, acc_4h:0.6996456980705261
EURCAD, acc_1h:0.2055632770061493, acc_4h:0.09010427445173264
EURCHF, acc_1h:0.15575113892555237, acc_4h:0.5468862652778625
EURGBP, acc_1h:0.3568339943885803, acc_4h:0.30247607827186584
EURJPY, acc_1h:0.08744162321090698, acc_4h:0.28508487343788147
EURNZD, acc_1h:0.171932652592659, acc_4h:0.28698331117630005
EURUSD, acc_1h:0.12198175489902496, acc_

GBPUSD, acc_1h:0.24883897602558136, acc_4h:0.33049386739730835
NZDJPY, acc_1h:0.41826650500297546, acc_4h:0.5686653852462769
NZDCAD, acc_1h:0.351372092962265, acc_4h:0.027773095294833183
NZDCHF, acc_1h:0.1713719367980957, acc_4h:0.7591124176979065
NZDUSD, acc_1h:0.22145836055278778, acc_4h:0.5359373092651367
USDCAD, acc_1h:0.5820502042770386, acc_4h:0.582563042640686
USDCHF, acc_1h:0.24529443681240082, acc_4h:0.26715201139450073
USDJPY, acc_1h:0.21465177834033966, acc_4h:0.29988834261894226
---- 2022-01-19 23:59:55.006196 ------ 
AUDCAD, acc_1h:0.27462688088417053, acc_4h:0.22483274340629578
AUDCHF, acc_1h:0.00758511945605278, acc_4h:0.7096555233001709
--------- Sell Position --------
0.65942006
AUDJPY, acc_1h:0.6833591461181641, acc_4h:0.4832710921764374
AUDNZD, acc_1h:0.02315555326640606, acc_4h:0.6939679980278015
AUDUSD, acc_1h:0.1807452291250229, acc_4h:0.415046751499176
CADCHF, acc_1h:0.4319879114627838, acc_4h:0.5297225117683411
CADJPY, acc_1h:0.07667320221662521, acc_4h:0.745756

EURAUD, acc_1h:0.1279192417860031, acc_4h:0.20809800922870636
EURCAD, acc_1h:0.27980607748031616, acc_4h:0.2976497411727905
EURCHF, acc_1h:0.7185231447219849, acc_4h:0.5804092288017273
EURGBP, acc_1h:0.18318593502044678, acc_4h:0.3103492259979248
EURJPY, acc_1h:0.3563738763332367, acc_4h:0.10488463193178177
EURNZD, acc_1h:0.358498752117157, acc_4h:0.4024556577205658
EURUSD, acc_1h:0.2047443687915802, acc_4h:0.3601823151111603
GBPAUD, acc_1h:0.3273443877696991, acc_4h:0.6048283576965332
GBPCAD, acc_1h:0.42481711506843567, acc_4h:0.3817524015903473
GBPCHF, acc_1h:0.2271641492843628, acc_4h:0.9839072227478027
GBPJPY, acc_1h:0.3645322620868683, acc_4h:0.1914713978767395
GBPNZD, acc_1h:0.31901663541793823, acc_4h:0.28837209939956665
GBPUSD, acc_1h:0.1612287163734436, acc_4h:0.5336688756942749
NZDJPY, acc_1h:0.6285547018051147, acc_4h:0.5798389911651611
NZDCAD, acc_1h:0.45459139347076416, acc_4h:0.3507002592086792
NZDCHF, acc_1h:0.10868970304727554, acc_4h:0.4557272791862488
NZDUSD, acc_1h:0

NZDCHF, acc_1h:0.41836968064308167, acc_4h:0.017272721976041794
NZDUSD, acc_1h:0.4727429449558258, acc_4h:0.14055393636226654
USDCAD, acc_1h:0.06493121385574341, acc_4h:0.3648144602775574
USDCHF, acc_1h:0.02592877671122551, acc_4h:0.47544917464256287
USDJPY, acc_1h:0.06822323054075241, acc_4h:0.25279295444488525
---- 2022-01-20 08:59:55.003302 ------ 
AUDCAD, acc_1h:0.17628268897533417, acc_4h:0.34880390763282776
AUDCHF, acc_1h:0.2587472200393677, acc_4h:0.2931795120239258
AUDJPY, acc_1h:0.231392040848732, acc_4h:0.0840381607413292
AUDNZD, acc_1h:0.18271537125110626, acc_4h:0.8282343745231628
AUDUSD, acc_1h:0.4901500940322876, acc_4h:0.24586261808872223
CADCHF, acc_1h:0.13419103622436523, acc_4h:0.2168360948562622
CADJPY, acc_1h:0.0055195922031998634, acc_4h:0.32793670892715454
CHFJPY, acc_1h:0.27281874418258667, acc_4h:0.3102894723415375
EURAUD, acc_1h:0.3997379541397095, acc_4h:0.3377358615398407
EURCAD, acc_1h:0.1983707845211029, acc_4h:0.02854177914559841
EURCHF, acc_1h:0.301233828

EURJPY, acc_1h:0.2255481779575348, acc_4h:0.2893430292606354
EURNZD, acc_1h:0.41670432686805725, acc_4h:0.14938804507255554
EURUSD, acc_1h:0.3078143298625946, acc_4h:0.3974460959434509
GBPAUD, acc_1h:0.2802733778953552, acc_4h:0.5705253481864929
GBPCAD, acc_1h:0.4991486668586731, acc_4h:0.4906221032142639
GBPCHF, acc_1h:0.18466626107692719, acc_4h:0.4729456901550293
GBPJPY, acc_1h:0.30057796835899353, acc_4h:0.31911495327949524
GBPNZD, acc_1h:0.3781571388244629, acc_4h:0.8453807234764099
GBPUSD, acc_1h:0.3180505037307739, acc_4h:0.2232285737991333
NZDJPY, acc_1h:0.15565474331378937, acc_4h:0.48469164967536926
NZDCAD, acc_1h:0.4275011122226715, acc_4h:0.294906884431839
NZDCHF, acc_1h:0.07708094269037247, acc_4h:0.08829044550657272
NZDUSD, acc_1h:0.27592864632606506, acc_4h:0.200811967253685
USDCAD, acc_1h:0.5502223372459412, acc_4h:0.8511841297149658
USDCHF, acc_1h:0.1753893792629242, acc_4h:0.7641799449920654
USDJPY, acc_1h:0.209614098072052, acc_4h:0.4962288439273834
---- 2022-01-20 1

...

---- 2022-01-21 19:59:55.013523 ------ 
AUDCAD, acc_1h:0.40726056694984436, acc_4h:0.11491348594427109
AUDCHF, acc_1h:0.13880257308483124, acc_4h:0.07668008655309677
AUDJPY, acc_1h:0.08979865908622742, acc_4h:0.34327301383018494
AUDNZD, acc_1h:0.26329004764556885, acc_4h:0.315738320350647
AUDUSD, acc_1h:0.21469418704509735, acc_4h:0.08978548645973206
CADCHF, acc_1h:0.2551432251930237, acc_4h:0.7895894646644592
CADJPY, acc_1h:0.10196417570114136, acc_4h:0.1512986719608307
CHFJPY, acc_1h:0.42191779613494873, acc_4h:0.22044993937015533
EURAUD, acc_1h:0.19853198528289795, acc_4h:0.9134096503257751
EURCAD, acc_1h:0.27125152945518494, acc_4h:0.3616263270378113
EURCHF, acc_1h:0.15661516785621643, acc_4h:0.6989715695381165
EURGBP, acc_1h:0.22043418884277344, acc_4h:0.4897156059741974
EURJPY, acc_1h:0.2835273742675781, acc_4h:0.1295125037431717
EURNZD, acc_1h:0.33310970664024353, acc_4h:0.412627637386322
EURUSD, acc_1h:0.384368360042572, acc_4h:0.12559129297733307
GBPAUD, acc_1h:0.2166426032781

GBPAUD, acc_1h:0.1036420464515686, acc_4h:0.14405661821365356
GBPCAD, acc_1h:0.06029864400625229, acc_4h:0.12186068296432495
GBPCHF, acc_1h:0.0009127476369030774, acc_4h:0.003701447509229183
--------- Sell Position --------
1.23793006
GBPJPY, acc_1h:0.12754566967487335, acc_4h:0.4082845151424408
GBPNZD, acc_1h:0.3652585446834564, acc_4h:0.30529555678367615
GBPUSD, acc_1h:0.03162837401032448, acc_4h:0.25398582220077515
NZDJPY, acc_1h:0.14679786562919617, acc_4h:0.2966426610946655
NZDCAD, acc_1h:0.1974497139453888, acc_4h:0.25814223289489746
NZDCHF, acc_1h:0.08703397959470749, acc_4h:0.4172147512435913
NZDUSD, acc_1h:0.26243096590042114, acc_4h:0.24995510280132294
USDCAD, acc_1h:0.2513747215270996, acc_4h:0.37196701765060425
USDCHF, acc_1h:0.01504114642739296, acc_4h:0.8435065746307373
USDJPY, acc_1h:0.13657483458518982, acc_4h:0.32326704263687134
---- 2022-01-22 00:59:55.004810 ------ 
AUDCAD, acc_1h:0.44159039855003357, acc_4h:0.2071295827627182
AUDCHF, acc_1h:0.014272813685238361, acc

USDCHF, acc_1h:0.05340715870261192, acc_4h:0.16976268589496613
USDJPY, acc_1h:0.04848945885896683, acc_4h:0.4206114709377289
---- 2022-01-22 04:59:55.014732 ------ 
AUDCAD, acc_1h:0.828112006187439, acc_4h:0.01342066377401352
AUDCHF, acc_1h:0.024364294484257698, acc_4h:0.6280083060264587
AUDJPY, acc_1h:0.15718655288219452, acc_4h:0.4892294406890869
AUDNZD, acc_1h:0.5137537717819214, acc_4h:0.6497328877449036
AUDUSD, acc_1h:0.6073758006095886, acc_4h:0.08627726882696152
CADCHF, acc_1h:0.11039029806852341, acc_4h:0.388578325510025
CADJPY, acc_1h:0.07404819875955582, acc_4h:0.262960821390152
CHFJPY, acc_1h:0.49015194177627563, acc_4h:0.14432582259178162
EURAUD, acc_1h:0.147757425904274, acc_4h:0.013365978375077248
EURCAD, acc_1h:0.5384416580200195, acc_4h:0.11427989602088928
EURCHF, acc_1h:0.33547350764274597, acc_4h:0.8389320373535156
EURGBP, acc_1h:0.7242623567581177, acc_4h:0.3095739483833313
EURJPY, acc_1h:0.29383331537246704, acc_4h:0.29558390378952026
EURNZD, acc_1h:0.28856906294822

EURJPY, acc_1h:0.3153254985809326, acc_4h:0.30071118474006653
EURNZD, acc_1h:0.41735053062438965, acc_4h:0.32726895809173584
EURUSD, acc_1h:0.3568349778652191, acc_4h:0.29054850339889526
GBPAUD, acc_1h:0.28405842185020447, acc_4h:0.7248077988624573
GBPCAD, acc_1h:0.4853856861591339, acc_4h:0.0658826231956482
GBPCHF, acc_1h:0.24059131741523743, acc_4h:0.23454049229621887
GBPJPY, acc_1h:0.2563766837120056, acc_4h:0.26598384976387024
GBPNZD, acc_1h:0.5027622580528259, acc_4h:0.6907750368118286
GBPUSD, acc_1h:0.3082946240901947, acc_4h:0.23107361793518066
NZDJPY, acc_1h:0.19261084496974945, acc_4h:0.5024757981300354
NZDCAD, acc_1h:0.21265514194965363, acc_4h:0.20319415628910065
NZDCHF, acc_1h:0.2587270140647888, acc_4h:0.36621949076652527
NZDUSD, acc_1h:0.2416786253452301, acc_4h:0.17252866923809052
USDCAD, acc_1h:0.6687489748001099, acc_4h:0.8295134902000427
USDCHF, acc_1h:0.1030648872256279, acc_4h:0.7753374576568604
USDJPY, acc_1h:0.2823750078678131, acc_4h:0.5581845045089722
---- 2022-

KeyboardInterrupt: 